In [10]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
import sys

In [11]:
model = load_model("gender_age_emotion_model.h5")
input_shape = (48, 48) 

In [12]:
# Initialize Mediapipe face detection
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

In [13]:
def preprocess_face(face):
    """Preprocess the face for model input."""
    if face is None or face.size == 0:
        return None  # Invalid face region
    face_resized = cv2.resize(face, input_shape)  # Resize to model input shape
    face_normalized = face_resized / 255.0  # Normalize pixel values
    face_expanded = np.expand_dims(face_normalized, axis=0)  # Add batch dimension
    return face_expanded

In [14]:
def live_gender_age_detection():
    """Real-time gender and age detection."""
    cap = cv2.VideoCapture(0)  # Start webcam
    try:
        with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
            while True:
                ret, frame = cap.read()
                if not ret:
                    print("Failed to capture frame from webcam. Exiting...")
                    break

                # Convert frame to RGB for Mediapipe
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = face_detection.process(rgb_frame)

                if results.detections:
                    for detection in results.detections:
                        # Get bounding box from Mediapipe detection
                        bboxC = detection.location_data.relative_bounding_box
                        h, w, _ = frame.shape
                        x1, y1 = int(bboxC.xmin * w), int(bboxC.ymin * h)
                        x2, y2 = int((bboxC.xmin + bboxC.width) * w), int((bboxC.ymin + bboxC.height) * h)

                        # Extract face from frame
                        face = frame[y1:y2, x1:x2]
                        face_processed = preprocess_face(face)

                        if face_processed is None:
                            continue  # Skip invalid face regions

                        # Make predictions
                        predictions = model.predict(face_processed)[0]
                        gender_pred = predictions[:2]  # Assuming first two outputs are gender
                        age_pred = predictions[2:]  # Assuming rest are age bins

                        # Interpret predictions
                        gender = "Male" if gender_pred[0] > gender_pred[1] else "Female"
                        predicted_age = int(np.dot(np.arange(101), age_pred))  # Weighted average for age

                        # Draw bounding box and predictions on frame
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(frame, f"Gender: {gender}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                        cv2.putText(frame, f"Age: {predicted_age}", (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

                # Display the frame
                cv2.imshow("Gender and Age Detection", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    print("Exiting...")
                    break
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        cap.release()
        cv2.destroyAllWindows()
        sys.exit(0)  # Ensure proper termination

# Run the detection
live_gender_age_detection()

An error occurred: in user code:

    File "C:\Users\TAREQ\anaconda3\lib\site-packages\keras\src\engine\training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\TAREQ\anaconda3\lib\site-packages\keras\src\engine\training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\TAREQ\anaconda3\lib\site-packages\keras\src\engine\training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\TAREQ\anaconda3\lib\site-packages\keras\src\engine\training.py", line 2283, in predict_step
        return self(x, training=False)
    File "C:\Users\TAREQ\anaconda3\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\TAREQ\anaconda3\lib\site-packages\keras\src\engine\input_spec.py", line 280, in assert_input_compatibility
        raise V

SystemExit: 0